In [2]:
import numpy as np
import pandas as pd
from math import ceil
import pickle
with open('datapath.txt') as f:
    datapath=f.readlines()[0].rstrip()

In [3]:
%ls $datapath

cliente_tabla.csv.zip  expfile.csv~            SplitFiles/   town_state.csv.zip
expfile.csv            producto_tabla.csv.zip  test.csv.zip  train.csv*


In [4]:
train=pd.read_csv(datapath+'train.csv',nrows=10,parse_dates=False)

In [5]:
dtypes=train.dtypes.to_dict()
dtypes['Semana']=np.uint8
dtypes

{'Agencia_ID': dtype('int64'),
 'Canal_ID': dtype('int64'),
 'Cliente_ID': dtype('int64'),
 'Demanda_uni_equil': dtype('int64'),
 'Dev_proxima': dtype('float64'),
 'Dev_uni_proxima': dtype('int64'),
 'Producto_ID': dtype('int64'),
 'Ruta_SAK': dtype('int64'),
 'Semana': numpy.uint8,
 'Venta_hoy': dtype('float64'),
 'Venta_uni_hoy': dtype('int64')}

In [6]:
yname='Demanda_uni_equil'

In [7]:
categoricals='Agencia_ID Canal_ID Ruta_SAK Cliente_ID Producto_ID'.split()

In [5]:
def addcounts(cnt,cnk,stcol):
    newcnt=[df_counts(cnk,stcol[i]) for i in xrange(len(stcol))]
    return [cnt[i].add(newcnt[i],axis='index',fill_value=0.0) for i in xrange(len(stcol))]

In [6]:
def df_counts(df,column):
    return df.groupby(column).size()

In [11]:
def s_stats(s):
    s=s['Demanda_uni_equil']
    ser=pd.Series()
    ser['count']=s.count()
    ser['sum']=s.sum()
    ser['sqsum']=(s**2).sum()
    return ser

In [8]:
statcolumns='Agencia_ID Canal_ID Ruta_SAK Producto_ID Semana'.split()

In [9]:
numcolumns='Venta_uni_hoy Venta_hoy Dev_uni_proxima Dev_proxima Demanda_uni_equil'.split()

In [7]:
def add_stats(one,two):
    df=pd.DataFrame(columns=one.columns)
    df.loc['count']=one.loc['count']+two.loc['count']
    df.loc['sum']=(one.loc['sum'] + two.loc['sum'])
    df.loc['sqsum']=(one.loc['squm'])
    return df
    

In [10]:
chunksz=100000
chunk=pd.read_csv(datapath+'train.csv',nrows=chunksz,dtype=dtypes,parse_dates=False,usecols='Cliente_ID Demanda_uni_equil'.split())

In [12]:
chunk.groupby('Cliente_ID').apply(s_stats)

,count,sum,sqsum
Cliente_ID,,,
14075,68,446,5958
14077,6,10,20
15766,23,131,917
16119,4,68,1350
16127,1,1,1
16505,8,25,85
16509,6,12,32
16755,8,69,955
17146,3,22,250


In [11]:
chunksz=100000
trainit=pd.read_csv(datapath+'train.csv',chunksize=chunksz,iterator=True,dtype=dtypes,parse_dates=False,usecols='Cliente_ID Demanda_uni_equil'.split())

In [12]:
stats='count mean max min'.split()

In [13]:
#%%time
ichunk=0
for chunk in trainit:
    if ichunk==0:
        fullcount=[df_counts(chunk,col) for col in statcolumns]
        floatstats=chunk[numcolumns].describe().loc[stats]
        #sample=chunk.sample(frac=samplerate)
    else:
        fullcount=addcounts(fullcount,chunk,statcolumns)
        floatstats=add_stats(floatstats,chunk[numcolumns].describe().loc[stats])
    if ichunk % 100 == 0:
        print(ichunk)
        print map(lambda x: x.shape[0], fullcount)
    ichunk+=1

0
[3, 4, 151, 287, 1]
100
[499, 9, 2744, 1546, 1]
200
[552, 9, 3013, 1625, 2]
300
[552, 9, 3249, 1663, 3]
400
[552, 9, 3413, 1698, 4]
500
[552, 9, 3487, 1730, 5]
600
[552, 9, 3554, 1753, 6]
700
[552, 9, 3599, 1782, 7]


In [17]:
with open('catcount.pkl','w') as f:
    pickle.dump((floatstats,fullcount), f)

In [5]:
%ls

catcount.pkl          Cliente_ID_stats.py  Feature Stats Generation.ipynb
Cliente_ID_Stats.pkl  datapath.txt


I don't seem to have enough memory for `Cliente_ID`, so I had to run it from a raw python script, which I called `Cliente_ID_stats.py`. The results are saved in `Cliente_ID_Stats.pkl`.